Popularity_model

In the following code, we open the file, create a mapping from a song ID to the number of times this song appears, and close the file. 

In [14]:
f = open('/Users/sinasaba/Desktop/MillionSongSubset/AdditionalFiles/kaggle/kaggle_visible_evaluation_triplets.txt', 'r')
song_to_count = dict() 
for line in f: 
    _, song, _ = line.strip().split('\t')
    if song in song_to_count: 
        song_to_count[song] += 1 
    else:
        song_to_count[song] = 1
        
f.close() 

we reorder the songs by decreasing popularity: 

In [15]:
songs_orderd = sorted(song_to_count.keys(), 
                     key = lambda s: song_to_count[s], 
                     reverse = True)

We will recommend the most popular songs to every user, but we must filter out songs already in the user’s library. Reopening the triplets file, we will create a map from user to songs they have listened to.

In [16]:
f = open('/Users/sinasaba/Desktop/MillionSongSubset/AdditionalFiles/kaggle/kaggle_visible_evaluation_triplets.txt', 'r')
user_to_songs = dict()
for line in f: 
    user, song, _ = line.strip().split('\t')
    if user in user_to_songs: 
        user_to_songs[user].add(song)
    else:
        user_to_songs[user] = set([song])
f.close()

For each user, we now have a list of songs: 

Ok, we now have the songs ordered by popularity, and listening history for each user. To produce our submission file, we’ll need to load the canonical ordering of users:

In [17]:
f = open('/Users/sinasaba/Desktop/MillionSongSubset/AdditionalFiles/kaggle/kaggle_users.txt', 'r')
canonical_users = map(lambda line: line.strip(), f.readlines()) 
f.close()

we now have the list of users in the order expected in the submission. The first two users are: 

In [18]:
canonical_users = list(canonical_users)

We are almost there, but we're missing one more thing. To reduce the size of submission files, we do not submit a list of song IDs such as SOSOUKN12A8C13AB79, but rather their index in the canonical list of songs.


Let's create the map from song ID to song index (for those unfamiliar with python, this line is even more magic than before).

In [20]:
g = open('/Users/sinasaba/Desktop/MillionSongSubset/AdditionalFiles/kaggle/kaggle_songs.txt', 'r')
song_to_index = dict(map(lambda line: line.strip().split(' '), g.readlines()))
g.close()

Finally, we are ready to create the submission file. For each user in the canonical list, recommend the songs in order of popularity, except those already in the user’s profile.

In [22]:
f = open('/Users/sinasaba/Desktop/MillionSongSubset/AdditionalFiles/kaggle/submission.txt', 'w')
for user in list(canonical_users):
    songs_to_recommend = []
    for song in songs_orderd:
        if len(songs_to_recommend) >= 500:
            break
        if not song in user_to_songs[user]:
            songs_to_recommend.append(song)
    # Transform song IDs to song indexes
    indices = map(lambda s: song_to_index[s],
                  songs_to_recommend)
    # Write line for that user
    f.write(' '.join(indices) + '\n')
    
f.close()

And that's it! We have our submission file: submission.txt